# 서울시 주요 공원 현황

In [49]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import folium
import json
import re
import requests
from urllib.parse import urlparse
import googlemaps
from tqdm import tqdm_notebook
%matplotlib inline
# 한글폰트 사용
mpl.rc('font', family='Malgun Gothic')
mpl.rc('axes', unicode_minus=False)

In [50]:
park_in_seoul = pd.read_csv('./data/서울시 주요 공원현황.csv', sep=',', encoding='euc-kr')
park_in_seoul.head(1)

,공원번호,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107,450395.554,126.990377,37.55014,http://parks.seoul.go.kr/template/sub/namsan.do


### 데이터 전처리

In [51]:
# '지역' 컬럼은 공원주소에서 추출할 것이기 때문에 받지 않습니다.
df = park_in_seoul[['공원명', '면적', '공원주소', 'Y좌표(WGS84)', 'X좌표(WGS84)']]
df.rename(columns={
    '면적': '공원면적', 'Y좌표(WGS84)': '위도', 'X좌표(WGS84)': '경도'
}, inplace=True)
df.head(3)

,공원명,공원면적,공원주소,위도,경도
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,서울특별시 중구 삼일대로 231(예장동),37.550140,126.990377
1,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원),37.540394,127.154779
2,서울대공원,9132690m²,경기도 과천시 대공원광장로 102,37.426449,127.019847


In [52]:
# 각 컬럼마다 결측치가 얼마나 있는지 확인
df.isnull().sum()

공원명     0
공원면적    3
공원주소    0
위도      1
경도      1
dtype: int64

In [53]:
# 면적과 좌표의 결측치를 우선 '0'(str)로 채웁니다.
df.fillna('0', inplace=True)
df.isnull().sum()

공원명     0
공원면적    0
공원주소    0
위도      0
경도      0
dtype: int64

#### '면적' 컬럼 전처리

In [54]:
print(df['공원면적'].tolist())

['2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡', '80683㎡', '9132690m²', '480994㎡', '2284085㎡', '8948.1㎡', '휴양 및 편익시설 : 평의자 7개 화장실 23.18㎡ 조경시설 - 은하폭포(높이 12m × 폭10m 발원지로부터 17.3m 폭 1.5 ~ 10m 수조 255㎡ 담수량 140t) - 산성폭포(높이 5m×폭 12m 수조 90㎡ 담수량 75t)- 명속의 벽 1개소(높이 5.4m) 교양시설 : 야외공연무대 102㎡ 관리시설 : 관리실 8.55㎡ 전기실 51.03㎡ 기계실 83.36㎡ 안내판 1개 공원 등 18본 휴양시설 : 벤취 12개 ', '6456㎥', '80309㎡ 배수지 시설용량 : 201.000㎥ 송수 펌프용량 : 직송급수계통 30000㎥/일 금호배수지계통 16400㎥/일 ', '1980.4㎡   ', '5197.7㎡ ', '11467㎡ ', '229539㎡', '15179.7 ㎡ ', '16734.60㎥ ', '61544㎡ 녹지대 : 40792㎡ 광 장 : 6600㎡ 기 타 : 14152㎡', '424106㎡ ', '560552㎡', '26696.8㎥ ', '75900㎡', '113021.7m2', '201779㎥ ', '258991㎡   ', '10420819.08㎡', '171294.1㎥', '총 15000㎡', '297926㎡', '411972㎡', '98470㎡', '2038741.3㎡', '603715.2㎡', '108120㎡', '29619.3㎥', '29843.8㎡ ', '126026㎡ ', '135002.7㎡ ', '384026㎡ ', '180000㎥', '416031㎡ ', '160720㎡', '2277252㎡ ', '19599.70㎥ ', '138596㎡ ', '185295㎥ ', '254648㎡', '29816.5㎡ ', '762642㎡ ', '664905㎡ ', '99599.9㎡ ', '5802㎡ ', '총 16500㎥ ', '585652㎥ ', 

In [55]:
# 대부분의 데이터가 split 했을때 맨 앞 덩어리가 총 면적인 것으로 보입니다.
tmp_list = []
for a in df['공원면적'].tolist():
    area = a.split(maxsplit=1)[0]           # 첫 공백으로 나눠 두 덩어리로 나누고 앞 덩어리를 면적으로 보고
    if area == '총':                        # 그 앞 덩어리가 '총' 일 경우
        area = a.split(maxsplit=1)[1]       # 두 번째 덩어리가 면적
    else:                                   # 그렇지 않으면
        area = a.split(maxsplit=1)[0]       # 앞 덩어리가 면적
    tmp_list.append(area)
print(tmp_list)

['2896887㎡', '80683㎡', '9132690m²', '480994㎡', '2284085㎡', '8948.1㎡', '휴양', '6456㎥', '80309㎡', '1980.4㎡', '5197.7㎡', '11467㎡', '229539㎡', '15179.7', '16734.60㎥', '61544㎡', '424106㎡', '560552㎡', '26696.8㎥', '75900㎡', '113021.7m2', '201779㎥', '258991㎡', '10420819.08㎡', '171294.1㎥', '15000㎡', '297926㎡', '411972㎡', '98470㎡', '2038741.3㎡', '603715.2㎡', '108120㎡', '29619.3㎥', '29843.8㎡', '126026㎡', '135002.7㎡', '384026㎡', '180000㎥', '416031㎡', '160720㎡', '2277252㎡', '19599.70㎥', '138596㎡', '185295㎥', '254648㎡', '29816.5㎡', '762642㎡', '664905㎡', '99599.9㎡', '5802㎡', '16500㎥ ', '585652㎥', '649709.6㎡', '72132㎡', '66027.8㎡', '265582㎡', '2088704.9㎡', '112070㎡', '993724㎡', '219167㎡', '1349556.4㎡', '1511198㎡', '1097565㎡', '328018㎡', '1447122㎡', '207790㎡', '1018689㎡', '954553㎡', '1667848㎡', '13224㎡', '130345㎡', '5332422㎡', '5107964㎡', '47832㎡', '213552㎡', '168099.87㎡', '359435.2㎡', '36660㎡', '380735㎡', '1807420.9㎡', '359342.8㎡', '632733㎡', '328018㎡', '6691885.3㎡', '285757㎡', '110407㎡', '29682.2㎡', '

- 결측치 대신 채워넣은 '0' 을 제외하고 인덱스 6의 데이터만 문제가 있어 보입니다.

In [56]:
# 인덱스 6의 데이터가 어떤 공원인지 확인
df.loc[6]

공원명                                         진관근린공원(구파발폭포)
공원면적    휴양 및 편익시설 : 평의자 7개 화장실 23.18㎡ 조경시설 - 은하폭포(높이 1...
공원주소                                    서울특별시 은평구 진관동 393
위도                                                 37.637
경도                                                126.919
Name: 6, dtype: object

In [57]:
# 진관근린공원 면적 검색결과를 해당 데이터에 덮어 씌웁니다.
# 아래에서 str 작업을 해줄 것이기 때문에 데이터를 우선 str 형태로 넣어줍니다.
tmp_list[6] = '983791'
print(tmp_list)

['2896887㎡', '80683㎡', '9132690m²', '480994㎡', '2284085㎡', '8948.1㎡', '983791', '6456㎥', '80309㎡', '1980.4㎡', '5197.7㎡', '11467㎡', '229539㎡', '15179.7', '16734.60㎥', '61544㎡', '424106㎡', '560552㎡', '26696.8㎥', '75900㎡', '113021.7m2', '201779㎥', '258991㎡', '10420819.08㎡', '171294.1㎥', '15000㎡', '297926㎡', '411972㎡', '98470㎡', '2038741.3㎡', '603715.2㎡', '108120㎡', '29619.3㎥', '29843.8㎡', '126026㎡', '135002.7㎡', '384026㎡', '180000㎥', '416031㎡', '160720㎡', '2277252㎡', '19599.70㎥', '138596㎡', '185295㎥', '254648㎡', '29816.5㎡', '762642㎡', '664905㎡', '99599.9㎡', '5802㎡', '16500㎥ ', '585652㎥', '649709.6㎡', '72132㎡', '66027.8㎡', '265582㎡', '2088704.9㎡', '112070㎡', '993724㎡', '219167㎡', '1349556.4㎡', '1511198㎡', '1097565㎡', '328018㎡', '1447122㎡', '207790㎡', '1018689㎡', '954553㎡', '1667848㎡', '13224㎡', '130345㎡', '5332422㎡', '5107964㎡', '47832㎡', '213552㎡', '168099.87㎡', '359435.2㎡', '36660㎡', '380735㎡', '1807420.9㎡', '359342.8㎡', '632733㎡', '328018㎡', '6691885.3㎡', '285757㎡', '110407㎡', '29682.2㎡

In [58]:
# 데이터의 필요치 않은 기호들을 제거
area_list = []
for b in tmp_list:
    area = re.sub('㎡|m²|m2|㎥', '', b)
    area_list.append(area)
print(area_list)

['2896887', '80683', '9132690', '480994', '2284085', '8948.1', '983791', '6456', '80309', '1980.4', '5197.7', '11467', '229539', '15179.7', '16734.60', '61544', '424106', '560552', '26696.8', '75900', '113021.7', '201779', '258991', '10420819.08', '171294.1', '15000', '297926', '411972', '98470', '2038741.3', '603715.2', '108120', '29619.3', '29843.8', '126026', '135002.7', '384026', '180000', '416031', '160720', '2277252', '19599.70', '138596', '185295', '254648', '29816.5', '762642', '664905', '99599.9', '5802', '16500 ', '585652', '649709.6', '72132', '66027.8', '265582', '2088704.9', '112070', '993724', '219167', '1349556.4', '1511198', '1097565', '328018', '1447122', '207790', '1018689', '954553', '1667848', '13224', '130345', '5332422', '5107964', '47832', '213552', '168099.87', '359435.2', '36660', '380735', '1807420.9', '359342.8', '632733', '328018', '6691885.3', '285757', '110407', '29682.2', '137759', '917190', '51470', '662543', '217946', '179666', '34955', '18083', '4835',

In [59]:
df['공원면적'] = area_list
df.head(10)

,공원명,공원면적,공원주소,위도,경도
0,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),37.5501,126.99
1,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),37.5404,127.155
2,서울대공원,9132690,경기도 과천시 대공원광장로 102,37.4264,127.02
3,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),37.5431,127.042
4,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),37.5718,126.879
5,광화문시민열린마당,8948.1,서울특별시 종로구 세종로 76-2,37.575,126.978
6,진관근린공원(구파발폭포),983791,서울특별시 은평구 진관동 393,37.637,126.919
7,중마루근린공원,6456,서울특별시 영등포구 영등포동2가 222,37.5198,126.911
8,응봉공원,80309,서울특별시 성동구 금호동1가 산1-72,37.5572,127.022
9,매화근린공원,1980.4,서울특별시 강서구 등촌동 630-14,37.5576,126.861


In [60]:
# dtype을 float로 설정
df = df.astype({
    '공원면적': float,
    '위도': float,
    '경도': float
    })
df.head(10)

,공원명,공원면적,공원주소,위도,경도
0,남산도시자연공원,2896887.0,서울특별시 중구 삼일대로 231(예장동),37.550140,126.990377
1,길동생태공원,80683.0,서울특별시 강동구 천호대로 1291(길동생태공원),37.540394,127.154779
2,서울대공원,9132690.0,경기도 과천시 대공원광장로 102,37.426449,127.019847
3,서울숲,480994.0,서울특별시 성동구 뚝섬로 273 (성수동1가),37.543072,127.041798
4,월드컵공원,2284085.0,서울특별시 마포구 하늘공원로 84(월드컵공원),37.571805,126.878907
5,광화문시민열린마당,8948.1,서울특별시 종로구 세종로 76-2,37.575045,126.978023
6,진관근린공원(구파발폭포),983791.0,서울특별시 은평구 진관동 393,37.637024,126.919216
7,중마루근린공원,6456.0,서울특별시 영등포구 영등포동2가 222,37.519832,126.911267
8,응봉공원,80309.0,서울특별시 성동구 금호동1가 산1-72,37.557228,127.021765
9,매화근린공원,1980.4,서울특별시 강서구 등촌동 630-14,37.557583,126.861150


- 인덱스 8의 주소처럼 정확하지 않은 주소가 들어가 있을 수 있으므로 구글맵을 통해 주소, 위도, 경도 정보를 다시 받아오겠습니다.

### 주소, 위도, 경도 정보 구글맵을 통해 다시 가져오기

In [61]:
key_fd = open('googlemapskey.txt', mode='r')
gmap_key = key_fd.read(100)
key_fd.close()

In [62]:
gmaps = googlemaps.Client(key=gmap_key)
gm = gmaps.geocode('서울대공원', language='ko')
gm

[{'address_components': [{'long_name': '１０２',
    'short_name': '１０２',
    'types': ['premise']},
   {'long_name': '대공원광장로',
    'short_name': '대공원광장로',
    'types': ['political', 'sublocality', 'sublocality_level_4']},
   {'long_name': '문원동',
    'short_name': '문원동',
    'types': ['political', 'sublocality', 'sublocality_level_2']},
   {'long_name': '과천시',
    'short_name': '과천시',
    'types': ['locality', 'political']},
   {'long_name': '경기도',
    'short_name': '경기도',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': '대한민국',
    'short_name': 'KR',
    'types': ['country', 'political']},
   {'long_name': '427-702',
    'short_name': '427-702',
    'types': ['postal_code']}],
  'formatted_address': '대한민국 경기도 과천시 문원동 대공원광장로 102',
  'geometry': {'location': {'lat': 37.4275247, 'lng': 127.0170252},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast': {'lat': 37.4288736802915,
     'lng': 127.0183741802915},
    'southwest': {'lat': 37.4261757197085, 'lng

In [63]:
gm[0]['formatted_address'], gm[0]['geometry']['location']['lat'], gm[0]['geometry']['location']['lng']

('대한민국 경기도 과천시 문원동 대공원광장로 102', 37.4275247, 127.0170252)

In [64]:
# 공원명으로 검색했더니 나오지 않는 것도 있어서 일단 공백을 리스트에 넣어줍니다.
lat_list = []; lng_list = []; addr_list = []
for name in tqdm_notebook(df['공원명'].tolist()):
    gm = gmaps.geocode(name, language='ko')
    if gm == []:
        addr_list.append(' ')
        lat_list.append(' ')
        lng_list.append(' ')
    else:
        addr_list.append(gm[0]['formatted_address'])
        gm_loc = gm[0]['geometry']['location']
        lat_list.append(gm_loc['lat'])
        lng_list.append(gm_loc['lng'])

In [65]:
tmp_df = pd.DataFrame({
    '이름': df['공원명'],
    '주소': addr_list,
    'lat': lat_list,
    'lng': lng_list
})
tmp_df.head(10)

,이름,주소,lat,lng
0,남산도시자연공원,,,
1,길동생태공원,대한민국 서울특별시 강동구 길동 산7-3 길동생태공원,37.541,127.156
2,서울대공원,대한민국 경기도 과천시 문원동 대공원광장로 102,37.4275,127.017
3,서울숲,대한민국 서울특별시 성동구 성수동1가 뚝섬로 273 서울숲공원 SEOUL FOREST,37.5444,127.037
4,월드컵공원,대한민국 서울특별시 마포구 성산2동 월드컵로 243-60 월드컵공원,37.5639,126.897
5,광화문시민열린마당,,,
6,진관근린공원(구파발폭포),대한민국 서울특별시 은평구 진관동 진관근린공원,37.6408,126.926
7,중마루근린공원,대한민국 서울특별시 영등포구 영등포동 영등포로53길 22 중마루공원,37.5199,126.911
8,응봉공원,대한민국 서울특별시 성동구 금호동1가 72 응봉공원,37.5572,127.022
9,매화근린공원,,,


- 위에서 확인했던 인덱스 8의 잘못된 주소 정보는 제대로 수정된 것으로 보여집니다.

#### 공원명으로 구글맵 검색을 했더니 제대로 나오지 않는 공원 수만 24개나 되었습니다.

In [66]:
len(tmp_df[tmp_df['주소'] == ' '])

24

#### 구글맵 검색으로 나온 결과는 쓰고 나오지 않은 공원들은 원래 데이터를 그대로 쓰겠습니다.

In [67]:
addr_list = []; lat_list = []; lng_list = []
for i in df.index:
    if tmp_df['주소'].tolist()[i] != ' ':                   # 구글맵 검색을 통해 제대로 나온 데이터는 그 결과를 리스트에 넣어줍니다.
        addr_list.append(tmp_df['주소'].tolist()[i])
        lat_list.append(tmp_df['lat'].tolist()[i])
        lng_list.append(tmp_df['lng'].tolist()[i])
    else:                                                   # 구글맵 검색을 통해 제대로 나오지 않는 공원들은 원래 데이터를 리스트에 넣어줍니다.
        addr_list.append(df['공원주소'].tolist()[i])
        lat_list.append(df['위도'].tolist()[i])
        lng_list.append(df['경도'].tolist()[i])

In [68]:
df['공원주소'] = addr_list
df['위도'] = lat_list
df['경도'] = lng_list
df.head(10)

,공원명,공원면적,공원주소,위도,경도
0,남산도시자연공원,2896887.0,서울특별시 중구 삼일대로 231(예장동),37.550140,126.990377
1,길동생태공원,80683.0,대한민국 서울특별시 강동구 길동 산7-3 길동생태공원,37.541021,127.156210
2,서울대공원,9132690.0,대한민국 경기도 과천시 문원동 대공원광장로 102,37.427525,127.017025
3,서울숲,480994.0,대한민국 서울특별시 성동구 성수동1가 뚝섬로 273 서울숲공원 SEOUL FOREST,37.544388,127.037442
4,월드컵공원,2284085.0,대한민국 서울특별시 마포구 성산2동 월드컵로 243-60 월드컵공원,37.563907,126.897317
5,광화문시민열린마당,8948.1,서울특별시 종로구 세종로 76-2,37.575045,126.978023
6,진관근린공원(구파발폭포),983791.0,대한민국 서울특별시 은평구 진관동 진관근린공원,37.640830,126.925584
7,중마루근린공원,6456.0,대한민국 서울특별시 영등포구 영등포동 영등포로53길 22 중마루공원,37.519928,126.911061
8,응봉공원,80309.0,대한민국 서울특별시 성동구 금호동1가 72 응봉공원,37.557222,127.021667
9,매화근린공원,1980.4,서울특별시 강서구 등촌동 630-14,37.557583,126.861150


### 공원주소 앞에 '대한민국' 이 붙은 데이터는 '대한민국' 제거 후 재설정

In [69]:
addr_list = []
for a in df['공원주소'].tolist():
    addr = a.split(maxsplit=1)[0]           # 공원주소를 맨 앞 공백을 기준으로 두 덩어리로 나눴을 때, 앞 덩어리를 addr로 놓고
    if addr == '대한민국':                  # addr == '대한민국' 이면
        addr = a.split(maxsplit=1)[1]       # 두 번째 덩어리를 addr에 대입(즉, '대한민국' 을 제외한 나머지가 주소로 재설정)
    else:                                   # '대한민국' 이 붙지 않은 주소 데이터는
        addr = a                            # 그냥 그대로 전부 주소로 들어간다.
    addr_list.append(addr)
df['공원주소'] = addr_list
df.head()

,공원명,공원면적,공원주소,위도,경도
0,남산도시자연공원,2896887.0,서울특별시 중구 삼일대로 231(예장동),37.550140,126.990377
1,길동생태공원,80683.0,서울특별시 강동구 길동 산7-3 길동생태공원,37.541021,127.156210
2,서울대공원,9132690.0,경기도 과천시 문원동 대공원광장로 102,37.427525,127.017025
3,서울숲,480994.0,서울특별시 성동구 성수동1가 뚝섬로 273 서울숲공원 SEOUL FOREST,37.544388,127.037442
4,월드컵공원,2284085.0,서울특별시 마포구 성산2동 월드컵로 243-60 월드컵공원,37.563907,126.897317


- '대한민국' 이 삭제된 것을 볼 수 있습니다.

### '공원주소' 데이터를 바탕으로 '구별' 컬럼 추가

In [70]:
addr_list = df['공원주소'].tolist()
region_list = []
for addr in addr_list:
    region = addr.split(maxsplit=2)[1]          # 공원주소를 앞에서부터 2개의 공백을 기준으로 3개의 덩어리로 나눴을 때 2번째 덩어리를 region에 대입
    region_list.append(region)

df['구별'] = region_list                        # '구별' 컬럼 추가
df.head()

,공원명,공원면적,공원주소,위도,경도,구별
0,남산도시자연공원,2896887.0,서울특별시 중구 삼일대로 231(예장동),37.550140,126.990377,중구
1,길동생태공원,80683.0,서울특별시 강동구 길동 산7-3 길동생태공원,37.541021,127.156210,강동구
2,서울대공원,9132690.0,경기도 과천시 문원동 대공원광장로 102,37.427525,127.017025,과천시
3,서울숲,480994.0,서울특별시 성동구 성수동1가 뚝섬로 273 서울숲공원 SEOUL FOREST,37.544388,127.037442,성동구
4,월드컵공원,2284085.0,서울특별시 마포구 성산2동 월드컵로 243-60 월드컵공원,37.563907,126.897317,마포구


### 공원주소가 서울특별시가 아닌 행 제거

In [71]:
addr_list = df['공원주소'].tolist()
for i, addr in enumerate(addr_list):
    region = addr.split(maxsplit=1)[0]
    if region != '서울특별시':
        df.drop(i, inplace=True)

df.reset_index(drop=True, inplace=True)         # 지워진 행이 있기 때문에 인덱스 초기화
df.head()

,공원명,공원면적,공원주소,위도,경도,구별
0,남산도시자연공원,2896887.0,서울특별시 중구 삼일대로 231(예장동),37.550140,126.990377,중구
1,길동생태공원,80683.0,서울특별시 강동구 길동 산7-3 길동생태공원,37.541021,127.156210,강동구
2,서울숲,480994.0,서울특별시 성동구 성수동1가 뚝섬로 273 서울숲공원 SEOUL FOREST,37.544388,127.037442,성동구
3,월드컵공원,2284085.0,서울특별시 마포구 성산2동 월드컵로 243-60 월드컵공원,37.563907,126.897317,마포구
4,광화문시민열린마당,8948.1,서울특별시 종로구 세종로 76-2,37.575045,126.978023,종로구


In [72]:
# 마지막으로 컬럼 순서를 변경해줍니다.
df = df[['구별', '공원명', '공원주소', '공원면적', '위도', '경도']]
df.head()

,구별,공원명,공원주소,공원면적,위도,경도
0,중구,남산도시자연공원,서울특별시 중구 삼일대로 231(예장동),2896887.0,37.550140,126.990377
1,강동구,길동생태공원,서울특별시 강동구 길동 산7-3 길동생태공원,80683.0,37.541021,127.156210
2,성동구,서울숲,서울특별시 성동구 성수동1가 뚝섬로 273 서울숲공원 SEOUL FOREST,480994.0,37.544388,127.037442
3,마포구,월드컵공원,서울특별시 마포구 성산2동 월드컵로 243-60 월드컵공원,2284085.0,37.563907,126.897317
4,종로구,광화문시민열린마당,서울특별시 종로구 세종로 76-2,8948.1,37.575045,126.978023


In [73]:
# 지금까지 처리한 데이터프레임을 csv 파일로 저장
df.to_csv('../input/Seoul/park_refactoring.csv', sep=',', encoding='utf8')

### 지도 위에 공원 위치 및 면적 표시하기

In [74]:
geo_path = '../input/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [76]:
sum_area = df.groupby(['구별'])[['공원면적']].sum()
sum_area.rename(columns={'공원면적': '총 면적'}, inplace=True)
sum_area.sort_values('총 면적', ascending=False).head()

,총 면적
구별,
노원구,12235699.30
관악구,10431453.08
중랑구,6937340.00
은평구,4963974.00
서초구,4439542.00


In [81]:
# 자치구별 공원들의 위치와 면적의 총합
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

map.choropleth(geo_data=geo_str,
                data=sum_area['총 면적'],
                columns=[sum_area.index, sum_area['총 면적']],
                fill_color='PuRd',
                key_on='feature.id')

for i in df.index:
    folium.Marker(
        location=[df['위도'][i], df['경도'][i]],
        popup=df['공원면적'][i], 
        tooltip=f"{df['구별'][i]} - {df['공원명'][i]}"
    ).add_to(map)
title_html = '<h3 align="center" style="font-size: 20px"><b>자치구별 공원들의 위치와 면적의 총합</b></h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

- 공원 면적의 총합은 노원, 관악, 중랑 순이었습니다.

In [82]:
count_park = df.groupby(['구별'])[['공원명']].count()
count_park.rename(columns={'공원명': '공원수'}, inplace=True)
count_park.sort_values('공원수', ascending=False).head(10)

,공원수
구별,
종로구,11
강서구,9
은평구,8
강동구,7
동작구,7
강남구,6
송파구,6
도봉구,6
마포구,5


In [83]:
# 자치구별 공원들의 위치와 공원의 수
map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5)

map.choropleth(geo_data=geo_str,
                data=count_park['공원수'],
                columns=[count_park.index, count_park['공원수']],
                fill_color='PuRd',
                key_on='feature.id')

for i in df.index:
    folium.Marker(
        location=[df['위도'][i], df['경도'][i]],
        popup=df['공원면적'][i], 
        tooltip=f"{df['구별'][i]} - {df['공원명'][i]}"
    ).add_to(map)
title_html = '<h3 align="center" style="font-size: 20px"><b>자치구별 공원들의 위치와 공원의 수</b></h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

- 자치구별 공원 수는 종로, 강서, 은평 순이었습니다.